# 감성 세기 예측 회귀 모델

In [215]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
# from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, cohen_kappa_score

from tqdm.auto import tqdm
import time
# from torch.utils.data import DataLoader, TensorDataset
# import torch.distributed as dist
# from torch.nn.parallel import DistributedDataParallel as DDP
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import random
import nltk
# from nltk.corpus import wordnet
from tf_keras.layers import Input, Dense, Dropout, Concatenate
from tf_keras.models import Model
from tf_keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 필요한 라이브러리 설치
# !pip install konlpy

import konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.utils import pprint

import os


## 데이터 전처리

csv 파일로 저장

In [216]:
# NLTK WordNet 다운로드
nltk.download('wordnet')
print("라이브러리 로드 완료")

# 데이터셋 로드
file_name = 'year4.csv'  # 감정 세기 데이터셋

# CSV 파일 로드
print("CSV 파일 로드 중...")
df = pd.read_csv(file_name, encoding='cp949')
print(f"데이터 로드 완료. 샘플 수: {len(df)}")

# 데이터 샘플 출력
print("데이터 샘플:")
print(df.head())

[nltk_data] Downloading package wordnet to /Users/m2021/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


라이브러리 로드 완료
CSV 파일 로드 중...
데이터 로드 완료. 샘플 수: 14606
데이터 샘플:
                     wav_id                        발화문     상황    1번 감정  \
0  5e258fd1305bcf3ad153a6a4           어, 청소 니가 대신 해 줘!  anger  Neutral   
1  5e258fe2305bcf3ad153a6a5         둘 다 청소 하기 싫어. 귀찮아.  anger  Neutral   
2  5e258ff5305bcf3ad153a6a6             둘 다 하기 싫어서 화내.  anger    Angry   
3  5e25902f305bcf3ad153a6a9                그럼 방세는 어떡해.  anger  Sadness   
4  5e27f90b5807b852d9e0157b  권태긴줄 알았는데 다른 사람이 생겼나보더라고.    sad  Sadness   

   1번 감정세기    2번 감정  2번 감정세기    3번 감정  3번 감정세기    4번 감정  4번 감정세기    5번 감정  \
0        0    Angry        1  Neutral        0  Neutral        0    Angry   
1        0    Angry        1  Neutral        0  Neutral        0    Angry   
2        1    Angry        1  Neutral        0    Angry        1    Angry   
3        1  Sadness        1  Sadness        1  Sadness        1  Sadness   
4        1  Sadness        1  Sadness        1  Sadness        2  Sadness   

   5번 감정세기  나이    성별  
0        1 

In [217]:
# Define emotion columns
emotion_labels = ['Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']

# Initialize a list to store processed data
final_data = []

# Process each row in the DataFrame
for _, row in df.iterrows():
    # Initialize emotion scores with zeros
    emotions = {emotion: 0 for emotion in emotion_labels}
    
    # Sum the scores for each emotion
    for i in range(1, 6):
        emotion = row[f'{i}번 감정']
        score = row[f'{i}번 감정세기']
        if emotion in emotions:
            emotions[emotion] += score

    # Append the processed data
    final_data.append({
        '발화문': row['발화문'],
        **emotions,
        '나이': row['나이'],
        '성별': row['성별']
    })

# Create the final DataFrame
final_df = pd.DataFrame(final_data)

# Convert emotion scores to integers
final_df[emotion_labels] = final_df[emotion_labels].astype(int)

# Display the final DataFrame
final_df.head(40)


,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,"어, 청소 니가 대신 해 줘!",0,0,0,2,0,0,0,27,male
1,둘 다 청소 하기 싫어. 귀찮아.,0,0,0,2,0,0,0,27,male
2,둘 다 하기 싫어서 화내.,0,0,0,4,0,0,0,27,male
3,그럼 방세는 어떡해.,0,0,5,0,0,0,0,27,male
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0,0,6,0,0,0,0,32,male
5,그냥 걷고 있어.,0,0,2,0,0,0,0,32,male
6,어. 고등학교 동창인데 이렇게 더럽게 쓸줄 몰랐어.,0,0,0,2,5,0,0,32,male
7,처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.,0,1,0,0,0,0,0,28,male
8,내가 애정 표현을 잘 못해서 자주 싸우긴 했어.,0,0,5,0,0,0,0,28,male
9,오늘 헤어졌어.,0,0,4,0,0,0,0,28,male


In [218]:
# 스케일링 및 데이터셋 분배

# Normalize emotion scores between 0 and 1
scaler = MinMaxScaler()
final_df[['Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']] = scaler.fit_transform(
    final_df[['Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']])

# # Prepare features and targets
# X = final_df['발화문']  # Features are the utterances
# y = final_df.drop(columns=['발화문'])  # Targets include emotion scores, age, and gender

# # Encode gender (if not already numeric)
# y['성별'] = y['성별'].map({'male': 0, 'female': 1})

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Display the size of each set
# print(f"Training set size: {len(X_train)}")
# print(f"Testing set size: {len(X_test)}")

# # Display a sample from the training data
# print("\nTraining data sample:")
# print(X_train.head())
# print(y_train.head())

In [219]:
final_df.to_csv('final_df.csv', index=False, encoding='utf-8-sig')

In [220]:
# Assuming final_df is your DataFrame with emotion columns
emotion_columns = ['Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']

# Initialize a dictionary to count the occurrences
dominant_emotion_count = {emotion: 0 for emotion in emotion_columns}

# Iterate over each row and find the dominant emotion
for _, row in final_df.iterrows():
    # Get the emotion with the maximum score for the current row
    dominant_emotion = row[emotion_columns].idxmax()
    
    # Increment the count for the dominant emotion
    dominant_emotion_count[dominant_emotion] += 1

# Convert the dictionary to a DataFrame for better visualization
dominant_emotion_df = pd.DataFrame(list(dominant_emotion_count.items()), columns=['Emotion', 'Count'])

# Display the result
print(dominant_emotion_df)

     Emotion  Count
0    Neutral    123
1  Happiness    370
2    Sadness   8172
3      Angry   3861
4    Disgust   1016
5   Surprise    113
6       Fear    951


In [221]:
# Calculate the mean for each emotion column
non_zero_means = final_df[['Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']].apply(lambda col: col[col != 0].mean())
emotion_mean = non_zero_means.mean()
emotion_mean

0.34102382168479234

## 불균형 처리

#### 일부 데이터 언더 샘플링 후 증강 (실패한 시도)

In [222]:
original_edited = final_df

In [223]:
original_edited['성별'] = original_edited['성별'].map({'male': 0, 'female': 1})
original_edited.head(40)

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,"어, 청소 니가 대신 해 줘!",0.0,0.000000,0.0,0.2,0.0,0.000000,0.0,27,0
1,둘 다 청소 하기 싫어. 귀찮아.,0.0,0.000000,0.0,0.2,0.0,0.000000,0.0,27,0
2,둘 다 하기 싫어서 화내.,0.0,0.000000,0.0,0.4,0.0,0.000000,0.0,27,0
3,그럼 방세는 어떡해.,0.0,0.000000,0.5,0.0,0.0,0.000000,0.0,27,0
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0.0,0.000000,0.6,0.0,0.0,0.000000,0.0,32,0
5,그냥 걷고 있어.,0.0,0.000000,0.2,0.0,0.0,0.000000,0.0,32,0
6,어. 고등학교 동창인데 이렇게 더럽게 쓸줄 몰랐어.,0.0,0.000000,0.0,0.2,0.5,0.000000,0.0,32,0
7,처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,28,0
8,내가 애정 표현을 잘 못해서 자주 싸우긴 했어.,0.0,0.000000,0.5,0.0,0.0,0.000000,0.0,28,0
9,오늘 헤어졌어.,0.0,0.000000,0.4,0.0,0.0,0.000000,0.0,28,0


In [224]:
# first_final_df = final_df

In [225]:
# 각 행에서 가장 높은 감정 컬럼 확인
final_df['Highest_Emotion'] = final_df[emotion_columns].idxmax(axis=1)

# 각 감정이 가장 높은 행의 수 계산
emotion_counts = final_df['Highest_Emotion'].value_counts()

# 결과 출력
print("각 감정이 가장 높은 행의 개수:")
print(emotion_counts)

각 감정이 가장 높은 행의 개수:
Highest_Emotion
Sadness      8172
Angry        3861
Disgust      1016
Fear          951
Happiness     370
Neutral       123
Surprise      113
Name: count, dtype: int64


##### 실패한 시도 (언더샘플링)

In [228]:
# 각 행에서 가장 높은 감정 컬럼 확인
final_df['Highest_Emotion'] = final_df[emotion_columns].idxmax(axis=1)

# 각 감정이 가장 높은 행의 수 계산
emotion_counts = final_df['Highest_Emotion'].value_counts()

# 결과 출력
print("각 감정이 가장 높은 행의 개수:")
print(emotion_counts)

각 감정이 가장 높은 행의 개수:
Highest_Emotion
Sadness      8172
Angry        3861
Disgust      1016
Fear          951
Happiness     370
Neutral       123
Surprise      113
Name: count, dtype: int64


In [229]:
df = first_final_df

In [234]:
random_seed = 42
df = df.sample(frac=1, random_state=random_seed).reset_index(drop=True)

# Sadness 이외의 감정 컬럼 값이 모두 0인 행의 인덱스 추출
emotion_columns = ['Neutral', 'Happiness', 'Angry', 'Disgust', 'Surprise', 'Fear']
zero_emotion_indices = df[(df[emotion_columns] == 0).all(axis=1)].index

# 4311개의 인덱스 무작위 선택
selected_indices = zero_emotion_indices.to_series().sample(n=4311, random_state=random_seed).values


# final_df에서 temp_unused와 동일한 행 삭제
final_df = df.drop(index=selected_indices).reset_index(drop=True)

# 결과 확인
print("temp_unused 데이터프레임:")
print(temp_unused.head())
print("\nfinal_df 데이터프레임 (업데이트됨):")
print(final_df.head())

temp_unused 데이터프레임:
                                     발화문  Neutral  Happiness  Sadness  Angry  \
0  할 수 있으면은 당연히 좋지. 근데 이직도 힘드니까. 잘 모르겠어.      0.0        0.0      0.7    0.0   
1                          나 결국 헤어지게 됐어.      0.0        0.0      0.7    0.0   
2                      밥맛이 없어. 나중에 먹어야지.      0.0        0.0      0.6    0.0   
3                        해피가 결국 세상을 떠났어.      0.0        0.0      0.7    0.0   
4                                권태기인가봐.      0.0        0.0      0.6    0.0   

   Disgust  Surprise  Fear  나이  성별 Highest_Emotion  
0      0.0       0.0   0.0  24   0         Sadness  
1      0.0       0.0   0.0  46   1         Sadness  
2      0.0       0.0   0.0  46   1         Sadness  
3      0.0       0.0   0.0  40   1         Sadness  
4      0.0       0.0   0.0  27   1         Sadness  

final_df 데이터프레임 (업데이트됨):
                                       발화문  Neutral  Happiness  Sadness  \
0      아니. 조금 예전부터 시름시름 아프더니 결국 세상을 떠나버렸네.      0.0   0.000000      0.7   
1    

In [235]:
# 각 행에서 가장 높은 감정 컬럼 확인
final_df['Highest_Emotion'] = final_df[emotion_columns].idxmax(axis=1)

# 각 감정이 가장 높은 행의 수 계산
emotion_counts = final_df['Highest_Emotion'].value_counts()

# 결과 출력
print("각 감정이 가장 높은 행의 개수:")
print(emotion_counts)

각 감정이 가장 높은 행의 개수:
Highest_Emotion
Angry        5272
Neutral      1721
Fear         1404
Disgust      1147
Happiness     609
Surprise      142
Name: count, dtype: int64


In [236]:
final_df = first_final_df

# 각 행에서 가장 높은 감정 컬럼 확인
final_df['Highest_Emotion'] = final_df[emotion_columns].idxmax(axis=1)

# 각 감정이 가장 높은 행의 수 계산
emotion_counts = final_df['Highest_Emotion'].value_counts()

# 결과 출력
print("각 감정이 가장 높은 행의 개수:")
print(emotion_counts)

각 감정이 가장 높은 행의 개수:
Highest_Emotion
Neutral      6032
Angry        5272
Fear         1404
Disgust      1147
Happiness     609
Surprise      142
Name: count, dtype: int64


#### 한글 패키지 사용하기 (실패한 시도)

In [237]:
konlpy.data.path

['/Users/m2021/konlpy_data',
 '/usr/share/konlpy_data',
 '/usr/local/share/konlpy_data',
 '/usr/lib/konlpy_data',
 '/usr/local/lib/konlpy_data',
 '/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/konlpy/data']

In [238]:
os.chdir('/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/konlpy/data')
os.getcwd()

!jar xvf open-korean-text-2.1.0.jar

java.io.FileNotFoundException: open-korean-text-2.1.0.jar (No such file or directory)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:213)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:152)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:106)
	at jdk.jartool/sun.tools.jar.Main.run(Main.java:394)
	at jdk.jartool/sun.tools.jar.Main.main(Main.java:1708)


In [239]:
# os.__file__

In [240]:
# with open(f"/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/konlpy/data/org/openkoreantext/processor/util") as f:
#     data = f.read()

# print(data[:50])

#### 데이터 부분 증강 (실패한 시도)

In [242]:
final_df

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별,Highest_Emotion,증강된 발화문
0,"어, 청소 니가 대신 해 줘!",0.0,0.0,0.0,0.2,0.0,0.0,0.0,27,0,Angry,"어, 청소 니가 대신 해 줘!"
1,둘 다 청소 하기 싫어. 귀찮아.,0.0,0.0,0.0,0.2,0.0,0.0,0.0,27,0,Angry,둘 다 청소 하기 싫어. 귀찮아.
2,둘 다 하기 싫어서 화내.,0.0,0.0,0.0,0.4,0.0,0.0,0.0,27,0,Angry,둘 다 하기 싫어서 화내.
3,그럼 방세는 어떡해.,0.0,0.0,0.5,0.0,0.0,0.0,0.0,27,0,Neutral,그럼 방세는 어떡해.
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0.0,0.0,0.6,0.0,0.0,0.0,0.0,32,0,Neutral,권태긴줄 알았는데 다른 사람이 생겼나보더라고.
...,...,...,...,...,...,...,...,...,...,...,...,...
14601,"아, 요즘 룸메랑 너무 자주 싸우게 돼.",0.0,0.0,0.0,0.4,0.2,0.0,0.0,35,0,Angry,"아, 요즘 룸메랑 너무 자주 싸우게 돼."
14602,"아, 룸메가 방을 너무 지저분하게 써. 음식물도 막 버리고.",0.0,0.0,0.0,0.4,0.2,0.0,0.0,35,0,Angry,"아, 룸메가 방을 너무 지저분하게 써. 음식물도 막 버리고."
14603,"뭐 화를 낸 것까진 아니지만, 한 달 전 쯤에 좀 확실하게 얘기를 해뒀거든. 근데 ...",0.0,0.0,0.1,0.4,0.2,0.0,0.0,35,0,Angry,"뭐 화를 낸 것까진 아니지만, 한 달 전 쯤에 좀 확실하게 얘기를 해뒀거든. 근데 ..."
14604,"어. 고등학교 동창인데, 같은 동네 오게 돼서 같이 룸메로 살게 됐지.",0.0,0.0,0.1,0.1,0.0,0.0,0.0,35,0,Angry,"어. 고등학교 동창인데, 같은 동네 오게 돼서 같이 룸메로 살게 됐지."


In [243]:
# Sadness 컬럼이 0이 아닌 행들을 추출하여 sadness_data에 저장
sadness_data = final_df[final_df['Sadness'] != 0].reset_index(drop=True)

# 나머지 행들을 추출하여 augment_df에 저장
augment_df = final_df[final_df['Sadness'] == 0].reset_index(drop=True)

# 결과 확인
print("Sadness가 0이 아닌 행들 (sadness_data):")
print(sadness_data.head())
print("\n나머지 행들 (augment_df):")
print(augment_df.head())


Sadness가 0이 아닌 행들 (sadness_data):
                          발화문  Neutral  Happiness  Sadness  Angry  Disgust  \
0                 그럼 방세는 어떡해.      0.0        0.0      0.5    0.0      0.0   
1   권태긴줄 알았는데 다른 사람이 생겼나보더라고.      0.0        0.0      0.6    0.0      0.0   
2                   그냥 걷고 있어.      0.0        0.0      0.2    0.0      0.0   
3  내가 애정 표현을 잘 못해서 자주 싸우긴 했어.      0.0        0.0      0.5    0.0      0.0   
4                    오늘 헤어졌어.      0.0        0.0      0.4    0.0      0.0   

   Surprise  Fear  나이  성별 Highest_Emotion                     증강된 발화문  
0       0.0   0.0  27   0         Neutral                 그럼 방세는 어떡해.  
1       0.0   0.0  32   0         Neutral   권태긴줄 알았는데 다른 사람이 생겼나보더라고.  
2       0.0   0.0  32   0         Neutral                   그냥 걷고 있어.  
3       0.0   0.0  28   0         Neutral  내가 애정 표현을 잘 못해서 자주 싸우긴 했어.  
4       0.0   0.0  28   0         Neutral                    오늘 헤어졌어.  

나머지 행들 (augment_df):
                             발화문  Neutral  

In [244]:
# 형태소 분석기 초기화
okt = Okt()

# 동의어 치환을 위한 사전 예시
synonym_dict = {
}

# # 동의어 치환 함수
# def replace_synonyms_korean(sentence, synonym_dict, probability=0.3):
#     # 형태소 분석
#     words = okt.pos(sentence, norm=True, stem=True)
#     new_words = []
    
#     for word, pos in words:
#         if pos in ['Noun', 'Verb', 'Adjective'] and random.random() < probability:
#             # 동의어 치환
#             if word in synonym_dict:
#                 new_word = random.choice(synonym_dict[word])
#                 new_words.append(new_word)
#             else:
#                 new_words.append(word)
#         else:
#             new_words.append(word)
    
#     return ' '.join(new_words)

# # '발화문' 컬럼에 데이터 증강 적용
# augment_df['증강된 발화문'] = augment_df['발화문'].apply(lambda x: replace_synonyms_korean(x, synonym_dict))

# # 결과의 처음 몇 줄을 확인
# augment_df[['발화문', '증강된 발화문']].head()


In [245]:
dominant_emotion_df = pd.DataFrame(list(dominant_emotion_count.items()), columns=['Emotion', 'Count'])
# Display the result
print(dominant_emotion_df)

     Emotion  Count
0    Neutral    123
1  Happiness    370
2    Sadness   8172
3      Angry   3861
4    Disgust   1016
5   Surprise    113
6       Fear    951


In [246]:
print(final_df.head())

                         발화문  Neutral  Happiness  Sadness  Angry  Disgust  \
0           어, 청소 니가 대신 해 줘!      0.0        0.0      0.0    0.2      0.0   
1         둘 다 청소 하기 싫어. 귀찮아.      0.0        0.0      0.0    0.2      0.0   
2             둘 다 하기 싫어서 화내.      0.0        0.0      0.0    0.4      0.0   
3                그럼 방세는 어떡해.      0.0        0.0      0.5    0.0      0.0   
4  권태긴줄 알았는데 다른 사람이 생겼나보더라고.      0.0        0.0      0.6    0.0      0.0   

   Surprise  Fear  나이  성별 Highest_Emotion                    증강된 발화문  
0       0.0   0.0  27   0           Angry           어, 청소 니가 대신 해 줘!  
1       0.0   0.0  27   0           Angry         둘 다 청소 하기 싫어. 귀찮아.  
2       0.0   0.0  27   0           Angry             둘 다 하기 싫어서 화내.  
3       0.0   0.0  27   0         Neutral                그럼 방세는 어떡해.  
4       0.0   0.0  32   0         Neutral  권태긴줄 알았는데 다른 사람이 생겼나보더라고.  


#### 불균형 해결을 위해 다른 자료 통합 가능성 탐색 (기타 증강 기법 적용하지 않음)

In [247]:
file_two = 'training_chunked_dialogue.xlsx'
file_three = 'val_chunked_dialogue.xlsx'

# EXCEL 파일 로드
print("EXCEL 파일 로드 중...")
df = pd.read_excel(file_two)
print(f"데이터 로드 완료. 샘플 수: {len(df)}")

# 데이터 샘플 출력
print("데이터 샘플:")
print(df.head())

EXCEL 파일 로드 중...


FileNotFoundError: [Errno 2] No such file or directory: 'training_chunked_dialogue.xlsx'

In [ ]:
# Create sets for 감정_소분류 and 감정_대분류
sub_emotions_set = set(df['감정_소분류'].dropna().unique())
big_emotions_set = set(df['감정_대분류'].dropna().unique())

print("\n감정_소분류 집합:", sub_emotions_set)
print("감정_대분류 집합:", big_emotions_set)

# Initialize an empty dictionary to store results
emotion_dict = {bigE: set() for bigE in big_emotions_set}

# Populate the dictionary
for bigE in big_emotions_set:
    for subE in sub_emotions_set:
        # Check if the current row has the given subE and bigE
        matching_rows = df[(df['감정_소분류'] == subE) & (df['감정_대분류'] == bigE)]
        if not matching_rows.empty:
            emotion_dict[bigE].add(subE)

# Convert sets to lists for more flexible usage later
emotion_dict = {key: list(value) for key, value in emotion_dict.items()}

# Display the resulting dictionary
print("\n감정_대분류와 감정_소분류 딕셔너리:")
for bigE, subE_list in emotion_dict.items():
    print(f"{bigE}: {subE_list}")


감정_소분류 집합: {'억울한', '환멸을 느끼는', '희생된', '불안', '편안한', '느긋', '외로운', '눈물이 나는', '신뢰하는', '분노', '배신당한', '스트레스 받는', '버려진', '좌절한', '짜증내는', '슬픔', '방어적인', '자신하는', '후회되는', '안달하는', '낙담한', '마비된', '신이 난', '우울한', '괴로워하는', '흥분', '회의적인', '상처', '실망한', '안도', '비통한', '충격 받은', '혼란스러운', '부끄러운', '당황', '질투하는', '혐오스러운', '남의 시선을 의식하는', '감사하는', '염세적인', '죄책감의', '걱정스러운', '노여워하는', '만족스러운', '툴툴대는', '가난한, 불우한', '두려운', '구역질 나는', '취약한', '고립된', '당혹스러운', '한심한', '초조한', '성가신', '조심스러운', '기쁨', '악의적인', '열등감'}
감정_대분류 집합: {'슬픔', '불안', '상처', '분노', '당황', '기쁨'}

감정_대분류와 감정_소분류 딕셔너리:
슬픔: ['슬픔', '환멸을 느끼는', '우울한', '눈물이 나는', '실망한', '비통한', '낙담한', '후회되는', '마비된', '염세적인']
불안: ['걱정스러운', '불안', '회의적인', '두려운', '취약한', '스트레스 받는', '당혹스러운', '혼란스러운', '초조한', '조심스러운']
상처: ['억울한', '희생된', '괴로워하는', '가난한, 불우한', '상처', '충격 받은', '고립된', '배신당한', '질투하는', '버려진']
분노: ['노여워하는', '방어적인', '툴툴대는', '구역질 나는', '분노', '안달하는', '성가신', '악의적인', '좌절한', '짜증내는']
당황: ['외로운', '열등감', '고립된', '혼란스러운', '부끄러운', '당황', '한심한', '혐오스러운', '남의 시선을 의식하는', '죄책감의']
기쁨: ['만족스러운', '흥분', '편안한', '느긋',

In [ ]:
df

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51625,51626,노년,남성,재정,만성질환 무,분노,성가신,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
51626,51627,노년,여성,재정,만성질환 무,불안,초조한,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
51627,51628,노년,여성,재정,만성질환 무,상처,희생된,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
51628,51629,노년,여성,대인관계,만성질환 무,불안,걱정스러운,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [ ]:
# Rename '사람문장1' to '발화문'
df.rename(columns={'사람문장1': '발화문'}, inplace=True)
df.rename(columns={'연령': '나이'}, inplace=True)

# Add new emotion columns with default value 0
for col in emotion_columns:
    df[col] = 0.0

# Define lists for specific emotions
list_disgust = ['한심한', '혐오스러운', '구역질 나는']
list_surprise = ['당황', '충격 받은']
list_sadness = ['고립된', '외로운', '슬픔', '염세적인', '실망한', '우울한', '환멸을 느끼는', '후회되는', 
                '낙담한', '눈물이 나는', '비통한', '마비된', '상처', '억울한', '버려진', '괴로워하는', 
                '질투하는', '희생된', '배신당한']
list_angry = ['노여워하는', '짜증내는', '분노', '좌절한', '성가신', '안달하는', '툴툴대는', 
              '악의적인', '방어적인']
list_fear = ['부끄러운', '죄책감의', '혼란스러운', '남의 시선을 의식하는', '열등감', '불안', '두려운', 
             '걱정스러운', '혼란스러운', '스트레스 받는', '회의적인', '초조한', '조심스러운', '취약한', 
             '당혹스러운']

# Update 'Happiness' where '감정_대분류' is '기쁨'
df.loc[df['감정_대분류'] == '기쁨', 'Happiness'] = emotion_mean
df.loc[df['감정_소분류'].isin(list_surprise), 'Surprise'] = emotion_mean
df.loc[df['감정_소분류'].isin(list_disgust), 'Disgust'] = emotion_mean
df.loc[df['감정_소분류'].isin(list_fear), 'Fear'] = emotion_mean
df.loc[df['감정_소분류'].isin(list_sadness), 'Sadness'] = emotion_mean
df.loc[df['감정_소분류'].isin(list_angry), 'Angry'] = emotion_mean


# Fill missing values with 0 (already initialized with 0.0)
# df[emotion_columns] = df[emotion_columns].fillna(0.0)

print(df.head())


   Unnamed: 0  나이  성별     상황키워드  신체질환 감정_대분류 감정_소분류  \
0           1  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
1           2  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
2           3  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
3           4  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
4           5  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   

                                                 발화문  \
0                          일은 왜 해도 해도 끝이 없을까? 화가 난다.   
1     이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.   
2  회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...   
3  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...   
4              얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.   

                                              시스템문장1  \
0                        많이 힘드시겠어요. 주위에 의논할 상대가 있나요?   
1           급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?   
2  회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...   
3  관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...   
4  무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...

In [ ]:
final_df

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별,Highest_Emotion
0,"어, 청소 니가 대신 해 줘!",0.0,0.0,0.0,0.2,0.0,0.0,0.0,27,0,Angry
1,둘 다 청소 하기 싫어. 귀찮아.,0.0,0.0,0.0,0.2,0.0,0.0,0.0,27,0,Angry
2,둘 다 하기 싫어서 화내.,0.0,0.0,0.0,0.4,0.0,0.0,0.0,27,0,Angry
3,그럼 방세는 어떡해.,0.0,0.0,0.5,0.0,0.0,0.0,0.0,27,0,Sadness
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0.0,0.0,0.6,0.0,0.0,0.0,0.0,32,0,Sadness
...,...,...,...,...,...,...,...,...,...,...,...
14601,"아, 요즘 룸메랑 너무 자주 싸우게 돼.",0.0,0.0,0.0,0.4,0.2,0.0,0.0,35,0,Angry
14602,"아, 룸메가 방을 너무 지저분하게 써. 음식물도 막 버리고.",0.0,0.0,0.0,0.4,0.2,0.0,0.0,35,0,Angry
14603,"뭐 화를 낸 것까진 아니지만, 한 달 전 쯤에 좀 확실하게 얘기를 해뒀거든. 근데 ...",0.0,0.0,0.1,0.4,0.2,0.0,0.0,35,0,Angry
14604,"어. 고등학교 동창인데, 같은 동네 오게 돼서 같이 룸메로 살게 됐지.",0.0,0.0,0.1,0.1,0.0,0.0,0.0,35,0,Sadness


In [ ]:
from_chunked_dial = df[['발화문', 'Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear', '나이', '성별']]

In [ ]:
# final_df = final_df.drop(columns=['Highest_Emotion'])

In [ ]:
from_chunked_dial

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,청년,여성
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,청년,여성
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,청년,여성
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,청년,여성
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,청년,여성
...,...,...,...,...,...,...,...,...,...,...
51625,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,노년,남성
51626,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,0.0,0.0,0.000000,0.000000,0.0,0.0,0.341024,노년,여성
51627,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,0.0,0.0,0.341024,0.000000,0.0,0.0,0.000000,노년,여성
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,0.0,0.0,0.000000,0.000000,0.0,0.0,0.341024,노년,여성


In [ ]:
age_groups = df['나이'].unique()
age_groups

array(['청년', '중년', '노년', '청소년'], dtype=object)

In [ ]:
def mean_age(a, b):
    age = (a + b) / 2
    return int(round(age))

teen = mean_age(13, 18)
adult = mean_age(19, 29)
midyear = mean_age(30, 49)
senior = mean_age(50, 80)


from_chunked_dial['성별'] = from_chunked_dial['성별'].map({'남성': 0, '여성': 1})
from_chunked_dial['나이'] = from_chunked_dial['나이'].map({'청소년': teen, '청년' : adult, '중년' : midyear, '노년': senior})


/var/folders/nc/35vy2ljn2pdfpcglx2fk1qn80000gn/T/ipykernel_66901/4226038590.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_chunked_dial['성별'] = from_chunked_dial['성별'].map({'남성': 0, '여성': 1})
/var/folders/nc/35vy2ljn2pdfpcglx2fk1qn80000gn/T/ipykernel_66901/4226038590.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_chunked_dial['나이'] = from_chunked_dial['나이'].map({'청소년': teen, '청년' : adult, '중년' : midyear, '노년': senior})


In [ ]:
from_chunked_dial

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,24,1
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,24,1
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,24,1
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,24,1
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,24,1
...,...,...,...,...,...,...,...,...,...,...
51625,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,0.0,0.0,0.000000,0.341024,0.0,0.0,0.000000,65,0
51626,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,0.0,0.0,0.000000,0.000000,0.0,0.0,0.341024,65,1
51627,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,0.0,0.0,0.341024,0.000000,0.0,0.0,0.000000,65,1
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,0.0,0.0,0.000000,0.000000,0.0,0.0,0.341024,65,1


In [ ]:
np.random.seed(42)

# Happiness 컬럼에서 0이 아닌 것 중 3491개 임의 추출
happiness_non_zero = from_chunked_dial[from_chunked_dial['Happiness'] != 0]
happiness_sample = happiness_non_zero.sample(n=3491, random_state=42)

disgust_non_zero = from_chunked_dial[from_chunked_dial['Disgust'] != 0]

surprise_non_zero = from_chunked_dial[from_chunked_dial['Surprise'] != 0]

# Fear 컬럼에서 0이 아닌 것 중 2910개 임의 추출
fear_non_zero = from_chunked_dial[from_chunked_dial['Fear'] != 0]
fear_sample = fear_non_zero.sample(n=2910, random_state=42)


selected = pd.concat([happiness_sample, disgust_non_zero, surprise_non_zero, fear_sample])

In [ ]:
original_edited = original_edited.drop(columns=['Highest_Emotion'])

In [ ]:
original_edited

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,"어, 청소 니가 대신 해 줘!",0.0,0.0,0.0,0.2,0.0,0.0,0.0,27,0
1,둘 다 청소 하기 싫어. 귀찮아.,0.0,0.0,0.0,0.2,0.0,0.0,0.0,27,0
2,둘 다 하기 싫어서 화내.,0.0,0.0,0.0,0.4,0.0,0.0,0.0,27,0
3,그럼 방세는 어떡해.,0.0,0.0,0.5,0.0,0.0,0.0,0.0,27,0
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0.0,0.0,0.6,0.0,0.0,0.0,0.0,32,0
...,...,...,...,...,...,...,...,...,...,...
14601,"아, 요즘 룸메랑 너무 자주 싸우게 돼.",0.0,0.0,0.0,0.4,0.2,0.0,0.0,35,0
14602,"아, 룸메가 방을 너무 지저분하게 써. 음식물도 막 버리고.",0.0,0.0,0.0,0.4,0.2,0.0,0.0,35,0
14603,"뭐 화를 낸 것까진 아니지만, 한 달 전 쯤에 좀 확실하게 얘기를 해뒀거든. 근데 ...",0.0,0.0,0.1,0.4,0.2,0.0,0.0,35,0
14604,"어. 고등학교 동창인데, 같은 동네 오게 돼서 같이 룸메로 살게 됐지.",0.0,0.0,0.1,0.1,0.0,0.0,0.0,35,0


In [ ]:
selected = selected[['발화문', 'Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear', '나이', '성별']]

In [ ]:
for_training = pd.concat([original_edited, selected], ignore_index=True)
for_training

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,"어, 청소 니가 대신 해 줘!",0.0,0.0,0.0,0.2,0.0,0.0,0.000000,27,0
1,둘 다 청소 하기 싫어. 귀찮아.,0.0,0.0,0.0,0.2,0.0,0.0,0.000000,27,0
2,둘 다 하기 싫어서 화내.,0.0,0.0,0.0,0.4,0.0,0.0,0.000000,27,0
3,그럼 방세는 어떡해.,0.0,0.0,0.5,0.0,0.0,0.0,0.000000,27,0
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0.0,0.0,0.6,0.0,0.0,0.0,0.000000,32,0
...,...,...,...,...,...,...,...,...,...,...
25404,새로운 아르바이트를 하는데 내가 일도 잘 못 하는 것 같고 민폐인 것 같아.,0.0,0.0,0.0,0.0,0.0,0.0,0.341024,24,0
25405,엄마 아빠와 나의 의견이 달라서 혼란스러워.,0.0,0.0,0.0,0.0,0.0,0.0,0.341024,16,1
25406,친구들끼리 술자리에 모이면 항상 싸움이 나서 스트레스받아.,0.0,0.0,0.0,0.0,0.0,0.0,0.341024,65,0
25407,학교에서 나를 괴롭히는 친구들이 나에게 부탁을 할 때마다 거절하지 못하겠어.,0.0,0.0,0.0,0.0,0.0,0.0,0.341024,16,0


In [260]:
for_training.head(50)

,발화문,Neutral,Happiness,Sadness,Angry,Disgust,Surprise,Fear,나이,성별
0,"어, 청소 니가 대신 해 줘!",0.0,0.000000,0.0,0.2,0.0,0.000000,0.0,27,0
1,둘 다 청소 하기 싫어. 귀찮아.,0.0,0.000000,0.0,0.2,0.0,0.000000,0.0,27,0
2,둘 다 하기 싫어서 화내.,0.0,0.000000,0.0,0.4,0.0,0.000000,0.0,27,0
3,그럼 방세는 어떡해.,0.0,0.000000,0.5,0.0,0.0,0.000000,0.0,27,0
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,0.0,0.000000,0.6,0.0,0.0,0.000000,0.0,32,0
5,그냥 걷고 있어.,0.0,0.000000,0.2,0.0,0.0,0.000000,0.0,32,0
6,어. 고등학교 동창인데 이렇게 더럽게 쓸줄 몰랐어.,0.0,0.000000,0.0,0.2,0.5,0.000000,0.0,32,0
7,처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,28,0
8,내가 애정 표현을 잘 못해서 자주 싸우긴 했어.,0.0,0.000000,0.5,0.0,0.0,0.000000,0.0,28,0
9,오늘 헤어졌어.,0.0,0.000000,0.4,0.0,0.0,0.000000,0.0,28,0


#### 훈련세트 테스트 세트 준비

In [ ]:
# Prepare features and targets
X = for_training['발화문']  # Features are the utterances
y = for_training.drop(columns=['발화문'])  # Targets include emotion scores, age, and gender

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the size of each set
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

# Display a sample from the training data
print("\nTraining data sample:")
print(X_train.head())
print(y_train.head())

Training set size: 20327
Testing set size: 5082

Training data sample:
24259                            나는 젊은 시절에 몇번의 사업실패를 겪었다네.
23403    회사에서 젊은 신입들이 가지고 있는 공과 사의 단호함을 보면 내심 부러우면서도 불안해져.
8714                                         장례를 해야될 것 같애.
4417               유기견 다큐멘터리 봤는데 사람들, 책임감 없는 사람들 진짜 너무하더라.
542                                    지금 혼자서 밖에 걸어다니고 있지.
Name: 발화문, dtype: object
       Neutral  Happiness  Sadness  Angry  Disgust  Surprise      Fear  나이  성별
24259      0.0        0.0      0.0    0.0      0.0       0.0  0.341024  65   0
23403      0.0        0.0      0.0    0.0      0.0       0.0  0.341024  40   0
8714       0.0        0.0      0.5    0.0      0.0       0.0  0.000000  29   0
4417       0.0        0.0      0.0    0.2      0.3       0.0  0.000000  24   1
542        0.0        0.0      0.4    0.0      0.0       0.0  0.000000  31   0


In [ ]:
X

In [ ]:
y

## 모델 구성

In [ ]:
# 토크나이저 및 모델 로드 (이전 코드와 동일)
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name) # 토크나이저 초기화
max_len = 128
bert_model = TFBertModel.from_pretrained(model_name)

# 데이터 인코딩 함수
def encode_data(dataframe, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []
    for text in dataframe:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf',
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

# 연결지어주기
# 데이터 인코딩
train_input_ids, train_attention_mask = encode_data(X_train, tokenizer)
val_input_ids, val_attention_mask = encode_data(X_test, tokenizer)

# 레이블 준비
train_emotions = y_train[['Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']].values
train_gender = y_train['성별'].values
train_labels = y_train['나이'].values

val_emotions = y_test[['Neutral', 'Happiness', 'Sadness', 'Angry', 'Disgust', 'Surprise', 'Fear']].values
val_gender = y_test['성별'].values
val_labels = y_test['나이'].values

# # Tokenize text data
# def tokenize_and_encode(texts):
#     return tokenizer(
#         texts.tolist(),
#         padding=True,
#         truncation=True,
#         max_length=128,
#         return_tensors="tf"
#     )

# # encodings = tokenizer(
# #     texts.to_list(),
# #     padding='max_length',
# #     truncation=True,
# #     max_length=128,
# #     return_tensors='tf'
# # )

# # Encode train and test data
# train_encodings = tokenize_and_encode(X_train)
# test_encodings = tokenize_and_encode(X_test)

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
X_train

24259                            나는 젊은 시절에 몇번의 사업실패를 겪었다네.
23403    회사에서 젊은 신입들이 가지고 있는 공과 사의 단호함을 보면 내심 부러우면서도 불안해져.
8714                                         장례를 해야될 것 같애.
4417               유기견 다큐멘터리 봤는데 사람들, 책임감 없는 사람들 진짜 너무하더라.
542                                    지금 혼자서 밖에 걸어다니고 있지.
                               ...                        
21575               친구가 다른 친구의 남자친구와 몰래 만나고 있었어. 너무 충격적이야.
5390                                     해피가 어젯밤에 세상을 떠났어.
860                                       룸메와 너무 자주 싸우게 돼.
15795                      이번 승진으로 개인 사무실을 갖게 되어 무척 만족스러워.
23654        형이랑 장난치다가 실수로 형의 눈에 상처를 냈어. 피가 너무 많이 나서 당황했어.
Name: 발화문, Length: 20327, dtype: object

In [ ]:
# 텐서 변환
train_emotions = tf.convert_to_tensor(train_emotions, dtype=tf.float32)
train_gender = tf.convert_to_tensor(train_gender, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

val_emotions = tf.convert_to_tensor(val_emotions, dtype=tf.float32)
val_gender = tf.convert_to_tensor(val_gender, dtype=tf.float32)
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.float32)


In [269]:
# class CustomBERTModel(tf.keras.Model):
#     def __init__(self, bert_model):
#         super(CustomBERTModel, self).__init__()
#         self.bert = bert_model
#         self.dropout = tf.keras.layers.Dropout(0.3)
#         self.emotion_dense = tf.keras.layers.Dense(7, activation='linear', name='emotion_regression')
#         self.gender_dense = tf.keras.layers.Dense(1, activation='sigmoid', name='gender_classification')
#         self.age_dense = tf.keras.layers.Dense(1, activation='linear', name='age_regression')

#     def call(self, inputs, training=False):
#         input_ids, attention_mask, emotions, gender = inputs
        
#         bert_output = self.bert(input_ids, attention_mask=attention_mask, training=training)
#         pooled_output = bert_output.pooler_output
        
#         x = self.dropout(pooled_output, training=training)
#         gender = tf.expand_dims(gender, axis=-1)
#         combined_input = tf.concat([x, emotions, gender], axis=1)
        
#         emotion_output = self.emotion_dense(combined_input)
#         gender_output = self.gender_dense(combined_input)
#         age_output = self.age_dense(combined_input)
        
#         return {
#             'emotion_regression': emotion_output,
#             'gender_classification': gender_output,
#             'age_regression': age_output
#         }

#     def train_step(self, data):
#         x, y = data
        
#         with tf.GradientTape() as tape:
#             y_pred = self(x, training=True)
#             emotion_loss = self.compiled_loss(y['emotion_regression'], y_pred['emotion_regression'], regularization_losses=self.losses)
#             gender_loss = self.compiled_loss(y['gender_classification'], y_pred['gender_classification'], regularization_losses=self.losses)
#             age_loss = self.compiled_loss(y['age_regression'], y_pred['age_regression'], regularization_losses=self.losses)
#             total_loss = emotion_loss + gender_loss + age_loss

#         trainable_vars = self.trainable_variables
#         gradients = tape.gradient(total_loss, trainable_vars)
        
#         self.optimizer.apply_gradients(zip(gradients, trainable_vars))
#         self.compiled_metrics.update_state(y, y_pred)
        
#         return {m.name: m.result() for m in self.metrics}

# # 모델 컴파일
# custom_model = CustomBERTModel(bert_model)
# custom_model.compile(
#     optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=3e-5),
#     loss={
#         'emotion_regression': 'mean_squared_error',
#         'gender_classification': 'binary_crossentropy',
#         'age_regression': 'mean_squared_error'
#     },
#     metrics={
#         'emotion_regression': 'mae',
#         'gender_classification': 'accuracy',
#         'age_regression': 'mae'
#     }
# )

# # 모델 훈련
# history = custom_model.fit(
#     x=[train_input_ids, train_attention_mask, train_emotions, train_gender],
#     y={
#         'emotion_regression': train_emotions,
#         'gender_classification': train_gender,
#         'age_regression': train_labels
#     },
#     validation_data=(
#         [val_input_ids, val_attention_mask, val_emotions, val_gender],
#         {
#             'emotion_regression': val_emotions,
#             'gender_classification': val_gender,
#             'age_regression': val_labels
#         }
#     ),
#     epochs=10,
#     batch_size=16,
#     verbose=1
# )

Epoch 1/10


ValueError: in user code:

    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/var/folders/nc/35vy2ljn2pdfpcglx2fk1qn80000gn/T/ipykernel_66901/3903446937.py", line 35, in train_step  *
        emotion_loss = self.compiled_loss(y['emotion_regression'], y_pred['emotion_regression'], regularization_losses=self.losses)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 252, in __call__  *
        self.build(y_pred)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/compile_utils.py", line 193, in build  *
        self._losses = self._conform_to_outputs(y_pred, self._losses)
    File "/opt/anacond

In [270]:
class CustomBERTModel(tf.keras.Model):
    def __init__(self, bert_model):
        super(CustomBERTModel, self).__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.emotion_dense = tf.keras.layers.Dense(7, activation='linear', name='emotion_regression')
        self.gender_dense = tf.keras.layers.Dense(1, activation='sigmoid', name='gender_classification')
        self.age_dense = tf.keras.layers.Dense(1, activation='linear', name='age_regression')

    def call(self, inputs, training=False):
        input_ids, attention_mask, emotions, gender = inputs
        
        bert_output = self.bert(input_ids, attention_mask=attention_mask, training=training)
        pooled_output = bert_output.pooler_output
        
        x = self.dropout(pooled_output, training=training)
        gender = tf.expand_dims(gender, axis=-1)
        combined_input = tf.concat([x, emotions, gender], axis=1)
        
        emotion_output = self.emotion_dense(combined_input)
        gender_output = self.gender_dense(combined_input)
        age_output = self.age_dense(combined_input)
        
        return {
            'emotion_regression': emotion_output,
            'gender_classification': gender_output,
            'age_regression': age_output
        }

# 모델 인스턴스 생성
custom_model = CustomBERTModel(bert_model)

# 모델 컴파일
custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss={
        'emotion_regression': 'mean_squared_error',
        'gender_classification': 'binary_crossentropy',
        'age_regression': 'mean_squared_error'
    },
    metrics={
        'emotion_regression': 'mae',
        'gender_classification': 'accuracy',
        'age_regression': 'mae'
    }
)

# 모델 구조 확인 (선택사항)
custom_model.build(input_shape=[(None, 128), (None, 128), (None, 7), (None,)])
custom_model.summary()

# 모델 훈련
history = custom_model.fit(
    x=[train_input_ids, train_attention_mask, train_emotions, train_gender],
    y={
        'emotion_regression': train_emotions,
        'gender_classification': train_gender,
        'age_regression': train_labels
    },
    validation_data=(
        [val_input_ids, val_attention_mask, val_emotions, val_gender],
        {
            'emotion_regression': val_emotions,
            'gender_classification': val_gender,
            'age_regression': val_labels
        }
    ),
    epochs=10,
    batch_size=16,
    verbose=1
)

ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, call your model on real tensor data (of the correct dtype).

The actual error from `call` is: Exception encountered when calling layer 'tf_bert_model_23' (type TFBertModel).

in user code:

    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/transformers/modeling_tf_utils.py", line 1182, in run_call_with_unpacked_inputs  *
        return func(self, **unpacked_inputs)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/transformers/models/bert/modeling_tf_bert.py", line 1209, in call  *
        outputs = self.bert(
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/base_layer.py", line 1005, in error_handler  *
        del filtered_tb
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/base_layer.py", line 1136, in __call__  *


In [272]:
class CustomBERTModel(tf.keras.Model):
    def __init__(self, bert_model):
        super(CustomBERTModel, self).__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.emotion_dense = tf.keras.layers.Dense(7, activation='linear', name='emotion_regression')
        self.gender_dense = tf.keras.layers.Dense(1, activation='sigmoid', name='gender_classification')
        self.age_dense = tf.keras.layers.Dense(1, activation='linear', name='age_regression')

    def call(self, inputs, training=False):
        input_ids, attention_mask, emotions, gender = inputs
        
        # 입력 데이터 타입 변환
        input_ids = tf.cast(input_ids, tf.int32)
        attention_mask = tf.cast(attention_mask, tf.int32)
        
        bert_output = self.bert(input_ids, attention_mask=attention_mask, training=training)
        pooled_output = bert_output.pooler_output
        
        x = self.dropout(pooled_output, training=training)
        gender = tf.expand_dims(gender, axis=-1)
        combined_input = tf.concat([x, emotions, gender], axis=1)
        
        emotion_output = self.emotion_dense(combined_input)
        gender_output = self.gender_dense(combined_input)
        age_output = self.age_dense(combined_input)
        
        return {
            'emotion_regression': emotion_output,
            'gender_classification': gender_output,
            'age_regression': age_output
        }

# 모델 인스턴스 생성
custom_model = CustomBERTModel(bert_model)

# 모델 컴파일
custom_model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=3e-5),
    loss={
        'emotion_regression': 'mean_squared_error',
        'gender_classification': 'binary_crossentropy',
        'age_regression': 'mean_squared_error'
    },
    metrics={
        'emotion_regression': 'mae',
        'gender_classification': 'accuracy',
        'age_regression': 'mae'
    }
)

# 모델 구조 확인
# 실제 데이터의 배치 크기를 사용하여 모델을 구축
batch_size = 16  # 또는 실제 사용하는 배치 크기
custom_model.build(input_shape=[
    (batch_size, 128),  # input_ids
    (batch_size, 128),  # attention_mask
    (batch_size, 7),    # emotions
    (batch_size,)       # gender
])
custom_model.summary()

# 모델 훈련
history = custom_model.fit(
    x=[train_input_ids, train_attention_mask, train_emotions, train_gender],
    y={
        'emotion_regression': train_emotions,
        'gender_classification': train_gender,
        'age_regression': train_labels
    },
    validation_data=(
        [val_input_ids, val_attention_mask, val_emotions, val_gender],
        {
            'emotion_regression': val_emotions,
            'gender_classification': val_gender,
            'age_regression': val_labels
        }
    ),
    epochs=10,
    batch_size=16,
    verbose=1
)

Model: "custom_bert_model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model_23 (TFBertMo  TFBaseModelOutputWithPo   92186880  
 del)                        olingAndCrossAttentions             
                             (last_hidden_state=(Non             
                             e, 128, 768),                       
                              pooler_output=(None, 7             
                             68),                                
                              past_key_values=None,              
                             hidden_states=None, att             
                             entions=None, cross_att             
                             entions=None)                       
                                                                 
 dropout_906 (Dropout)       multiple                  0         
                                              

TypeError: in user code:

    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1149, in train_step  *
        self._validate_target_and_loss(y, loss)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1102, in _validate_target_and_loss  *
        if self.loss and y is None:
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/typing.py", line 1912, in __instancecheck__
        val = getattr_static(instance, attr)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/inspect.py", line 1854, in getattr_static
        instance_result = _check_i

In [273]:
class CustomBERTModel(tf.keras.Model):
    def __init__(self, bert_model):
        super(CustomBERTModel, self).__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.emotion_dense = tf.keras.layers.Dense(7, activation='linear', name='emotion_regression')
        self.gender_dense = tf.keras.layers.Dense(1, activation='sigmoid', name='gender_classification')
        self.age_dense = tf.keras.layers.Dense(1, activation='linear', name='age_regression')

    def call(self, inputs, training=False):
        input_ids, attention_mask, emotions, gender = inputs
        
        # 입력 데이터 타입 변환
        input_ids = tf.cast(input_ids, tf.int32)
        attention_mask = tf.cast(attention_mask, tf.int32)
        
        bert_output = self.bert(input_ids, attention_mask=attention_mask, training=training)
        pooled_output = bert_output.pooler_output
        
        x = self.dropout(pooled_output, training=training)
        gender = tf.expand_dims(gender, axis=-1)
        combined_input = tf.concat([x, emotions, gender], axis=1)
        
        emotion_output = self.emotion_dense(combined_input)
        gender_output = self.gender_dense(combined_input)
        age_output = self.age_dense(combined_input)
        
        return {
            'emotion_regression': emotion_output,
            'gender_classification': gender_output,
            'age_regression': age_output
        }

    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            emotion_loss = self.compiled_loss['emotion_regression'](y['emotion_regression'], y_pred['emotion_regression'])
            gender_loss = self.compiled_loss['gender_classification'](y['gender_classification'], y_pred['gender_classification'])
            age_loss = self.compiled_loss['age_regression'](y['age_regression'], y_pred['age_regression'])
            total_loss = emotion_loss + gender_loss + age_loss

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)
        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        self.compiled_metrics['emotion_regression'].update_state(y['emotion_regression'], y_pred['emotion_regression'])
        self.compiled_metrics['gender_classification'].update_state(y['gender_classification'], y_pred['gender_classification'])
        self.compiled_metrics['age_regression'].update_state(y['age_regression'], y_pred['age_regression'])
        
        return {m.name: m.result() for m in self.metrics}

# 모델 인스턴스 생성
custom_model = CustomBERTModel(bert_model)

# 모델 컴파일
custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss={
        'emotion_regression': 'mean_squared_error',
        'gender_classification': 'binary_crossentropy',
        'age_regression': 'mean_squared_error'
    },
    metrics={
        'emotion_regression': 'mae',
        'gender_classification': 'accuracy',
        'age_regression': 'mae'
    }
)

# 모델 훈련
history = custom_model.fit(
    x=[train_input_ids, train_attention_mask, train_emotions, train_gender],
    y={
        'emotion_regression': train_emotions,
        'gender_classification': train_gender,
        'age_regression': train_labels
    },
    validation_data=(
        [val_input_ids, val_attention_mask, val_emotions, val_gender],
        {
            'emotion_regression': val_emotions,
            'gender_classification': val_gender,
            'age_regression': val_labels
        }
    ),
    epochs=10,
    batch_size=16,
    verbose=1
)

Epoch 1/10


TypeError: in user code:

    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/dlthonv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/var/folders/nc/35vy2ljn2pdfpcglx2fk1qn80000gn/T/ipykernel_66901/111251126.py", line 39, in train_step  *
        emotion_loss = self.compiled_loss['emotion_regression'](y['emotion_regression'], y_pred['emotion_regression'])

    TypeError: 'LossesContainer' object is not subscriptable


#### 실행 성공, 결과 확인 실패

In [275]:
class CustomBERTModel(tf.keras.Model):
    def __init__(self, bert_model):
        super(CustomBERTModel, self).__init__()
        self.bert = bert_model
        self.dropout = tf.keras.layers.Dropout(0.3)
        self.emotion_dense = tf.keras.layers.Dense(7, activation='linear', name='emotion_regression')
        self.gender_dense = tf.keras.layers.Dense(1, activation='sigmoid', name='gender_classification')
        self.age_dense = tf.keras.layers.Dense(1, activation='linear', name='age_regression')

    def call(self, inputs, training=False):
        input_ids, attention_mask, emotions, gender = inputs
        
        # 입력 데이터 타입 변환
        input_ids = tf.cast(input_ids, tf.int32)
        attention_mask = tf.cast(attention_mask, tf.int32)
        
        bert_output = self.bert(input_ids, attention_mask=attention_mask, training=training)
        pooled_output = bert_output.pooler_output
        
        x = self.dropout(pooled_output, training=training)
        gender = tf.expand_dims(gender, axis=-1)
        combined_input = tf.concat([x, emotions, gender], axis=1)
        
        emotion_output = self.emotion_dense(combined_input)
        gender_output = self.gender_dense(combined_input)
        age_output = self.age_dense(combined_input)
        
        return {
            'emotion_regression': emotion_output,
            'gender_classification': gender_output,
            'age_regression': age_output
        }

    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y, y_pred)
        
        return {m.name: m.result() for m in self.metrics}

# 모델 인스턴스 생성
custom_model = CustomBERTModel(bert_model)

# 모델 컴파일
custom_model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=3e-5),
    loss={
        'emotion_regression': 'mean_squared_error',
        'gender_classification': 'binary_crossentropy',
        'age_regression': 'mean_squared_error'
    },
    metrics={
        'emotion_regression': 'mae',
        'gender_classification': 'accuracy',
        'age_regression': 'mae'
    }
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5', 
    monitor='val_loss', 
    save_best_only=True
)

# 모델 훈련
history = custom_model.fit(
    x=[train_input_ids, train_attention_mask, train_emotions, train_gender],
    y={
        'emotion_regression': train_emotions,
        'gender_classification': train_gender,
        'age_regression': train_labels
    },
    validation_data=(
        [val_input_ids, val_attention_mask, val_emotions, val_gender],
        {
            'emotion_regression': val_emotions,
            'gender_classification': val_gender,
            'age_regression': val_labels
        }
    ),
    epochs=50,  # 최대 에폭 수 증가. Early Stopping이 적절한 시점에 훈련을 멈추도록
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# 학습 곡선 그리기
def plot_learning_curves(history):
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18))

    # 손실 곡선
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Emotion Regression MAE
    ax2.plot(history.history['emotion_regression_mae'], label='Training Emotion MAE')
    ax2.plot(history.history['val_emotion_regression_mae'], label='Validation Emotion MAE')
    ax2.set_title('Emotion Regression MAE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MAE')
    ax2.legend()

    # Gender Classification Accuracy
    ax3.plot(history.history['gender_classification_accuracy'], label='Training Gender Accuracy')
    ax3.plot(history.history['val_gender_classification_accuracy'], label='Validation Gender Accuracy')
    ax3.set_title('Gender Classification Accuracy')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Accuracy')
    ax3.legend()

    plt.tight_layout()
    plt.show()

# 학습 곡선 출력
plot_learning_curves(history)

# 최적의 에폭 출력
best_epoch = np.argmin(history.history['val_loss']) + 1
print(f"Best epoch: {best_epoch}")

Epoch 1/10
  23/1271 [..............................] - ETA: 3:02:50 - loss: 1166.9954 - age_regression_loss: 1165.9387 - emotion_regression_loss: 0.3594 - gender_classification_loss: 0.6971 - age_regression_mae: 31.0564 - emotion_regression_mae: 0.4726 - gender_classification_accuracy: 0.5380

KeyboardInterrupt: 

In [ ]:
# 차원 확인
# print(tf.shape(input_ids))  # 출력: (배치 크기, 입력 시퀀스 길이)
# print(tf.shape(attention_mask))  # 출력: (배치 크기, 입력 시퀀스 길이)

In [ ]:
# 입력 데이터 형태 확인
print("Input IDs shape:", train_input_ids.shape)  # 예시 출력: (배치 크기, 시퀀스 길이)
print("Attention Mask shape:", train_attention_mask.shape)  # 예시 출력: (배치 크기, 시퀀스 길이)
print("Emotions shape:", train_emotions.shape)  # 예시 출력: (배치 크기, 7)
print("Gender shape:", train_gender.shape)  # 예시 출력: (배치 크기, 2)
